In [8]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

import numpy as np
np.random.seed(0)
import random
import json
import pandas as pd
from datetime import datetime, timedelta

import pickle
import requests
from urllib.parse import urlencode
from sklearn.preprocessing import LabelEncoder
import json

# Visualisation
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from plotnine import ggplot, aes, geom_bar, theme_minimal, labs

from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from tqdm import tqdm, tqdm_notebook
from sklearn.metrics import accuracy_score, precision_score


In [9]:
df = pd.read_csv('C:/Users/Win11/Downloads/20240509_030938_45245_4dsx5.csv')
df_2 = pd.read_csv('C:/Users/Win11/Downloads/20240509_051907_83264_4dsx5.csv')
df = pd.concat([df, df_2], ignore_index = True)
df = df[df['matuirity_day'] == 2]

print(df.shape)
df.head(4)


(38611, 56)


,#account_id,register_time,matuirity_day,paying_matuirity,minutes_to_play,minutes_to_payment,minutes_to_pay_since_first_game,pay_matuirity,payments_to_date,payments_last_14d,...,win_amount_last_14d,win_amount_last_7d,win_money_rate_sequence,max_bet,past_disputer,new_disputer_30d,local_time_before_6,local_time_6_to_12,local_time_12_to_18,local_time_after_18
0,10207496,2024-01-02 00:33:46.000,2,2,0.0,3,3.0,2,2,2,...,16.39,16.39,"[0.57,null,null]",1.00,0,0,0.0,0.0,0.0,0.0
3,10208626,2024-01-02 03:05:04.000,2,2,0.0,4,4.0,2,3,3,...,35.07,35.07,"[0.49,null,null]",0.75,0,0,0.0,533.0,0.0,136.0
6,10208812,2024-01-02 03:27:23.000,2,2,0.0,26,26.0,2,2,2,...,16.80,16.80,"[0.67,null,null]",0.20,0,0,0.0,0.0,0.0,0.0
9,10209165,2024-01-02 04:13:48.000,2,2,0.0,4,4.0,2,5,5,...,137.15,137.15,"[0.25,0.36,0.3]",7.40,0,0,206.0,456.0,0.0,0.0


In [10]:
# REMOVE FEATURES AND DUMMIFY SEQUENCES 


def data_processing(df): 

    df['max_payment']           = df['max_payment']     / df['spent_to_date']
    df['max_daily_spent']       = df['max_daily_spent'] / df['spent_to_date']
    df['day_1_count_payment']   = (df['daily_payment_history_count'].apply(lambda x: x.replace('[', '').replace(']', '').split(',')[0])).astype(float)
    df['day_1_total_payment']   = (df['daily_spent_history_count'].apply(lambda x: x.replace('[', '').replace(']', '').split(',')[0])).astype(float)
    # df['d2_3_total_payment']    = (df['daily_spent_history_count'].apply(lambda x: x.replace('[', '').split(',')[1] + x.replace('[', '').split(',')[2])).astype(float)

    df['max_games_daily_games'] = df['max_games_daily_games'] / df['games_to_date']
    df['daily_games_first_1d']  = df['daily_games_first_1d']  / df['games_to_date']

    df['games_to_date']         = df['local_time_before_6'] + df['local_time_6_to_12'] + df['local_time_12_to_18'] + df['local_time_after_18'] 
    df['local_time_before_6']   = df['local_time_before_6'] / df['games_to_date']
    df['local_time_6_to_12']    = df['local_time_6_to_12']  / df['games_to_date']
    df['local_time_12_to_18']   = df['local_time_12_to_18'] / df['games_to_date']
    df['local_time_after_18']   = df['local_time_after_18'] / df['games_to_date']
    
    df['win_spent_ratio']       = df['win_amount_to_date']  / (df['spent_to_date']+0.0001)
    
    # Retention 
    df['full_[retention]']      = df['retention_sequence'].apply(lambda x: x[-2])
    df['full_pay_[retention]']  = df['daily_payment_history_binary'].apply(lambda x: x[-2])

    return df 
df = data_processing(df)

# Base Cols
cols = [
        # 'paying_matuirity', 'spent_to_date', 'spent_last_14d',  'spent_last_7d', 'max_daily_spent'
        'paying_matuirity', 'spent_to_date','max_daily_spent'
        
        # , 'max_payment', 'payments_last_14d', 'payments_last_7d',  'payments_to_date',  'minutes_to_payment', 'minutes_to_pay_since_first_game'
        , 'max_payment', 'payments_to_date',  'minutes_to_payment', 'minutes_to_pay_since_first_game'
        
        # , 'games_last_14d', 'games_last_7d', 'games_to_date', 'max_games_daily_games', 'daily_games_first_14d', 'daily_games_first_7d', 'daily_games_first_1d', 'minutes_to_play'
        , 'games_to_date', 'max_games_daily_games', 'daily_games_first_1d', 'minutes_to_play'
        
        # , 'money_games_to_date', 'bet_money_to_date', 'max_bet',  'money_games_last_14d', 'money_games_last_7d'
        , 'money_games_to_date', 'bet_money_to_date', 'max_bet'

        # , 'win_amount_to_date',  'win_amount_last_14d', 'win_amount_last_7d'
        , 'win_amount_to_date'
        
        , 'minutes_to_withdraw_since_first_payment', 'withdrawn_attempted_ammount_to_date', 'withdrawn_attempt_to_date'
        
        , 'local_time_before_6', 'local_time_6_to_12', 'local_time_12_to_18', 'local_time_after_18'
        ]
columns_of_interest = ['retention_sequence', 'days_betting_money_sequence', 'days_50_usd_bet_money_sequence', 'daily_payment_history_binary']  # Add additional columns here if needed
def truncate_list(lst):
    return lst[-14:]
for column in columns_of_interest:
    df[column] = df[column].apply(truncate_list)

# II. Execute the dummification
columns_info = {'retention_sequence': 'playing_days_'
                # , 'money_bet_sequence': 'money_bet_'
                , 'daily_payment_history_binary': 'payer_'
                # , 'daily_payment_history_count': 'pay_'
                , 'days_betting_money_sequence': 'days_bet_'
                , 'days_50_usd_bet_money_sequence': 'days_gt50_bet_'
                # , 'province': 'province_'
                }  # Add additional columns and identifiers here if needed

dummy_variable_list = []  # List to store dummy variables generated for each column

for column, identifier in columns_info.items():
    # Calculate sum of 'new_disputer_30d' for each unique value in the column
    sum_disputer = df.groupby(column)['new_disputer_30d'].sum()
    # Filter unique values where sum is >= 12
    valid_values = sum_disputer[sum_disputer >= 15].index.tolist()
    # Generate dummy variables for valid values
    dummy_variables = pd.get_dummies(df[column].loc[df[column].isin(valid_values)], prefix=identifier).astype(int)
    dummy_variable_list.append(dummy_variables)
    df = pd.concat([df, dummy_variables], axis=1)

# Concatenate all dummy variables into a single DataFrame
final_dummy_variables = pd.concat(dummy_variable_list, axis=1)
cols = cols + list(final_dummy_variables.columns)

# III. Can remove extra items
items_to_remove = ['working_hours_bets_to_date', 'minutes_to_withdraw_since_first_payment', 'withdrawn_attempted_ammount_to_date', 'withdrawn_attempt_to_date', 'province']
cols = [item for item in cols if item not in items_to_remove]
final_dummy_variables.columns


Index(['playing_days__[1,0,0]', 'playing_days__[1,0,1]',
       'playing_days__[1,1,0]', 'playing_days__[1,1,1]', 'payer__[1,0,0]',
       'payer__[1,0,1]', 'payer__[1,1,0]', 'payer__[1,1,1]',
       'days_bet__[1,0,0]', 'days_bet__[1,0,1]', 'days_bet__[1,1,0]',
       'days_bet__[1,1,1]', 'days_gt50_bet__[0,0,0]', 'days_gt50_bet__[1,0,0]',
       'days_gt50_bet__[1,0,1]', 'days_gt50_bet__[1,1,0]',
       'days_gt50_bet__[1,1,1]'],
      dtype='object')

In [11]:
df = df[df['day_1_total_payment'] < 500]

In [12]:

num_cols = [
            'spent_to_date', 'max_daily_spent', 'max_payment', 'payments_to_date', 'minutes_to_payment', 'minutes_to_pay_since_first_game', 
            'games_to_date', 'max_games_daily_games', 'daily_games_first_1d', 'minutes_to_play', 'money_games_to_date', 'bet_money_to_date', 'max_bet',
            'local_time_before_6', 'local_time_6_to_12', 'local_time_12_to_18', 'local_time_after_18',
            'win_spent_ratio', 'day_1_count_payment', 'day_1_total_payment'
            ]

def quantile_binning(df, columns, num_quantiles):

    # Get the qauntile for each column. 

    bins_dict = {}
    quantile_columns = {}
    
    for column in columns:
        # Calculate quantile bins
        quantile_labels, quantile_bins = pd.qcut(df[column], num_quantiles, retbins=True, duplicates='drop')
        
        # Remove duplicate values from bin edges
        quantile_bins = sorted(set(quantile_bins))
        
        # Apply binning to the column
        new_column_name = f"{column}_quantile"
        df[new_column_name] = pd.cut(df[column], bins=quantile_bins, labels=quantile_bins[:-1], include_lowest=True).astype(float)
        
        # Store the bins for each column
        bins_dict[column] = quantile_bins
        
        # Store the quantile column names
        quantile_columns[column] = new_column_name
    
    return df, bins_dict, quantile_columns

def quantile_dummies(df, column, quantile_bins):

    # Create dummy variables for each quantile
    for i in range(1, len(quantile_bins)):
        n = round(quantile_bins[i], 3)
        dummy_name = f"{column}_gt_{n}_true"
        df[dummy_name] = (df[column] > quantile_bins[i-1]).astype(int)

    return df

def quantile_dummies_all(df, num_cols, quantile_columns):

    for column in num_cols:
        quantile_column = quantile_columns.get(column)
        if quantile_column:
            quantile_bins = sorted(set(df[quantile_column]))
            quantile_bins = [x for x in quantile_bins if pd.notna(x)]
            df = quantile_dummies(df, column, quantile_bins)
    
    return df

df, bins_dict, quantile_columns = quantile_binning(df, num_cols, num_quantiles = 100)
df = quantile_dummies_all(df, num_cols, quantile_columns)


In [13]:
sequence_cols = [col for col in df.columns if col.endswith(']')]
quantile_col_ok = [col[:-5] for col in df.columns if col.endswith('_true')]
df.columns = [col[:-5] if col.endswith('_true') else col for col in df.columns]

cols_2 = list(quantile_col_ok) + list(sequence_cols)
cols_2


['spent_to_date_gt_2.99',
 'spent_to_date_gt_3.0',
 'spent_to_date_gt_4.0',
 'spent_to_date_gt_5.0',
 'spent_to_date_gt_6.0',
 'spent_to_date_gt_7.0',
 'spent_to_date_gt_8.0',
 'spent_to_date_gt_9.0',
 'spent_to_date_gt_10.0',
 'spent_to_date_gt_12.0',
 'spent_to_date_gt_13.0',
 'spent_to_date_gt_14.0',
 'spent_to_date_gt_15.0',
 'spent_to_date_gt_16.109',
 'spent_to_date_gt_18.0',
 'spent_to_date_gt_19.99',
 'spent_to_date_gt_20.0',
 'spent_to_date_gt_22.0',
 'spent_to_date_gt_23.0',
 'spent_to_date_gt_25.0',
 'spent_to_date_gt_27.0',
 'spent_to_date_gt_29.99',
 'spent_to_date_gt_30.0',
 'spent_to_date_gt_32.0',
 'spent_to_date_gt_34.99',
 'spent_to_date_gt_36.0',
 'spent_to_date_gt_40.0',
 'spent_to_date_gt_42.0',
 'spent_to_date_gt_45.0',
 'spent_to_date_gt_50.0',
 'spent_to_date_gt_55.0',
 'spent_to_date_gt_60.0',
 'spent_to_date_gt_65.0',
 'spent_to_date_gt_70.0',
 'spent_to_date_gt_79.99',
 'spent_to_date_gt_89.0',
 'spent_to_date_gt_98.0',
 'spent_to_date_gt_104.0',
 'spent_to_d

### Univariate rules

In [14]:

grouped_dfs = []
for col in cols_2: 
    grouped = df.groupby(col)['new_disputer_30d'].agg(['sum', 'count', 'mean']).reset_index()
    grouped = grouped.rename(columns={grouped.columns[0]: 'C_1'})
    grouped['C_1'] = grouped['C_1'].apply(lambda x: col + '_' + str(int(x)))
    grouped_dfs.append(grouped)

final_df = pd.concat(grouped_dfs)
final_df.insert(1, 'C_2', np.nan)
final_df.insert(2, 'C_3', np.nan)
final_df['C_1'] = final_df['C_1'].str.replace('_true_', '_')
final_df.reset_index(drop = True, inplace = True)


In [15]:
print(final_df.shape)
final_df.sort_values(by = 'mean', ascending = False).head(40)

(1938, 6)


,C_1,C_2,C_3,sum,count,mean
136,max_payment_gt_0.107_0,NaN,NaN,1,1,1.000000
233,payments_to_date_gt_18.0_1,NaN,NaN,24,659,0.036419
89,spent_to_date_gt_455.71_1,NaN,NaN,27,769,0.035111
1173,bet_money_to_date_gt_481.575_1,NaN,NaN,13,383,0.033943
138,max_payment_gt_0.139_0,NaN,NaN,13,385,0.033766
87,spent_to_date_gt_300.42_1,NaN,NaN,35,1149,0.030461
140,max_payment_gt_0.164_0,NaN,NaN,22,774,0.028424
85,spent_to_date_gt_225.0_1,NaN,NaN,42,1532,0.027415
1117,bet_money_to_date_gt_145.3_1,NaN,NaN,21,766,0.027415
231,payments_to_date_gt_14.0_1,NaN,NaN,30,1109,0.027051


In [16]:
selected_cases = final_df[final_df['sum'] >= 12]
relevants = list(set(list(selected_cases['C_1'].str.replace(r'_\d+$', '', regex=True))))
print(len(relevants))
relevants

969


['local_time_before_6_gt_0.455',
 'bet_money_to_date_gt_78.9',
 'max_games_daily_games_gt_0.466',
 'bet_money_to_date_gt_56.9',
 'local_time_after_18_gt_0.176',
 'bet_money_to_date_gt_26.6',
 'max_daily_spent_gt_0.467',
 'max_daily_spent_gt_0.521',
 'max_payment_gt_0.845',
 'bet_money_to_date_gt_119.1',
 'minutes_to_pay_since_first_game_gt_2.0',
 'minutes_to_payment_gt_5.0',
 'local_time_6_to_12_gt_0.078',
 'bet_money_to_date_gt_25.3',
 'bet_money_to_date_gt_524.97',
 'day_1_count_payment_gt_4.0',
 'minutes_to_payment_gt_834.36',
 'games_to_date_gt_145.0',
 'local_time_after_18_gt_0.233',
 'max_daily_spent_gt_0.573',
 'minutes_to_pay_since_first_game_gt_129.0',
 'local_time_after_18_gt_0.346',
 'minutes_to_pay_since_first_game_gt_1427.0',
 'local_time_after_18_gt_0.673',
 'games_to_date_gt_371.0',
 'max_daily_spent_gt_0.638',
 'money_games_to_date_gt_364.0',
 'local_time_after_18_gt_0.189',
 'games_to_date_gt_3294.4',
 'payments_to_date_gt_8.0',
 'local_time_12_to_18_gt_0.459',
 'win_s

In [17]:
import itertools
combis = list(itertools.combinations(relevants, 2))

# Extract the substrings before '_gt_' pattern from each tuple
prefixes = [[item.split('_gt_')[0] for item in pair] for pair in combis]

# Find the unique prefixes
unique_prefixes = set(prefixes[0])

# Initialize a list to store the filtered data
column_combinations = []

# Iterate over the data
for pair in combis:
    prefix1, prefix2 = [item.split('_gt_')[0] for item in pair]
    # Check if the prefixes are different
    if prefix1 != prefix2 or prefix1 not in unique_prefixes:
        column_combinations.append(pair)

column_combinations


[('local_time_before_6_gt_0.455', 'bet_money_to_date_gt_78.9'),
 ('local_time_before_6_gt_0.455', 'max_games_daily_games_gt_0.466'),
 ('local_time_before_6_gt_0.455', 'bet_money_to_date_gt_56.9'),
 ('local_time_before_6_gt_0.455', 'local_time_after_18_gt_0.176'),
 ('local_time_before_6_gt_0.455', 'bet_money_to_date_gt_26.6'),
 ('local_time_before_6_gt_0.455', 'max_daily_spent_gt_0.467'),
 ('local_time_before_6_gt_0.455', 'max_daily_spent_gt_0.521'),
 ('local_time_before_6_gt_0.455', 'max_payment_gt_0.845'),
 ('local_time_before_6_gt_0.455', 'bet_money_to_date_gt_119.1'),
 ('local_time_before_6_gt_0.455', 'minutes_to_pay_since_first_game_gt_2.0'),
 ('local_time_before_6_gt_0.455', 'minutes_to_payment_gt_5.0'),
 ('local_time_before_6_gt_0.455', 'local_time_6_to_12_gt_0.078'),
 ('local_time_before_6_gt_0.455', 'bet_money_to_date_gt_25.3'),
 ('local_time_before_6_gt_0.455', 'bet_money_to_date_gt_524.97'),
 ('local_time_before_6_gt_0.455', 'day_1_count_payment_gt_4.0'),
 ('local_time_before

In [18]:
# Assuming df is your DataFrame
# df.columns = [col[:-5] if col.endswith('_true') else col for col in df.columns]

# Initialize an empty list to store filtered DataFrames
filtered_dfs = []

# Iterate over each combination
for combination in tqdm(column_combinations):
    # Group by the current combination of columns
    grouped = df.groupby(list(combination))['new_disputer_30d'].agg(['sum', 'count', 'mean']).reset_index()
    # print(grouped)
    # Apply the filter condition
    filtered_grouped = grouped[(grouped['sum'] >= 15)]
    # Append the filtered DataFrame to the list
    filtered_dfs.append(filtered_grouped)

# Concatenate the filtered DataFrames
result_df = pd.concat(filtered_dfs)

# Print the concatenated DataFrame
result_df


  2%|▏         | 8868/462951 [00:43<33:12, 227.84it/s]  